In [1]:
!pip install Sastrawi

     |████████████████████████████████| 215kB 2.8MB/s 


In [2]:
import numpy as np
import pandas as pd
import pickle
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import sys
import os
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
os.environ['KERAS_BACKEND']='theano' # Why theano why not
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import gensim
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
plt.switch_backend('agg')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Using Theano backend.


##TEXT PREPROCESSING

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def case_folding(data):
  data = data.lower()
  return data

In [0]:
def remove_punctuation(data):
  data = data.translate(str.maketrans('','',string.punctuation))
  return data

In [0]:
def remove_numbers(data):
  data = re.sub(r'\d+','',data)
  return data

In [0]:
def remove_whitespace(data):
  data = data.strip()
  return data

In [0]:
def tokenizing(data):
  data = word_tokenize(data)
  return data

In [0]:
list_stopwords = set(stopwords.words('indonesian'))
def stopwords(data):
  hasil_list = []
  for i in data:
    if i not in list_stopwords and len(i)>=3 and len(i)<=12 and i not in hasil_list:
      hasil_list.append(i)
  data = hasil_list
  return data


In [0]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming(data):
  data_akhir = []
  for i in data:
    kata = stemmer.stem(i)
    data_akhir.append(kata)
  return data_akhir

#Inisialisasi Variabel

In [0]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

#READING DATA

In [13]:
df = pd.read_csv('drive/My Drive/Skripsi/Programs/Datasets/data_akhir.csv')
df = df.dropna()
df = df.reset_index(drop=True)
print('Shape of dataset ',df.shape)
print(df.columns)
print('No. of unique classes',len(set(df['Kelas'])))

Shape of dataset  (4366, 2)
Index(['Konten', 'Kelas'], dtype='object')
No. of unique classes 2


In [14]:
#Merubah Label menjadi nilai 0 atau 1
macronum=sorted(set(df['Kelas']))
macro_to_id = dict((note, number) for number, note in enumerate(macronum))
def fun(i):
    return macro_to_id[i]

df['Kelas']=df['Kelas'].apply(fun)
print(df['Kelas'])

0       1
1       0
2       1
3       0
4       1
       ..
4361    0
4362    1
4363    1
4364    1
4365    1
Name: Kelas, Length: 4366, dtype: int64


In [17]:
hasil_preprocessing = []
labels = []
i = 0

for idx in range(df.Konten.shape[0]): #df.Konten.shape[0]
  text = BeautifulSoup(df.Konten[idx])
  text = str(text.get_text())
  hasil_casefolding = case_folding(text)
  hasil_punctuation = remove_punctuation(hasil_casefolding)
  hasil_numbers = remove_numbers(hasil_punctuation)
  hasil_whitespace = remove_whitespace(hasil_numbers)
  hasil_tokenizing = tokenizing(hasil_whitespace)
  hasil_stopword = stopwords(hasil_tokenizing)
  hasil_stemming = stemming(hasil_stopword)
  hasil_preprocessing.append(hasil_stemming)
  print(i)
  i= i+1
  
  
for idx in df['Kelas']:
  labels.append(idx)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [0]:
df['Tokens'] = hasil_preprocessing
df['Kelas'] = labels
df['Text_Final'] = [' '.join(sen) for sen in hasil_preprocessing]

df = df[['Konten','Text_Final','Tokens','Kelas']]

In [19]:
df.head()

,Konten,Text_Final,Tokens,Kelas
0,: “audzu billahi minas syaithonir rojiim… ta...,audzu billahi mas syaithonir rojiim parkir ged...,"[audzu, billahi, mas, syaithonir, rojiim, park...",1
1,berdasarkan hasil penelusuran yang dilakukan...,dasar hasil telusur anggota grup forum anti fi...,"[dasar, hasil, telusur, anggota, grup, forum, ...",0
2,": “itu siapa bro??? presiden indonesia bro,” ...",bro presiden indonesia unggah akun facebook ed...,"[bro, presiden, indonesia, unggah, akun, faceb...",1
3,: akun facebook edelwies cleo atau @edelwies.c...,akun facebook edelwies cleo edelwiescleo ungga...,"[akun, facebook, edelwies, cleo, edelwiescleo,...",0
4,yang digunakan oleh sumber. selengkapnya di b...,sumber lengkap makhluk india gencar hina islam...,"[sumber, lengkap, makhluk, india, gencar, hina...",1


In [0]:
data_train, data_test = train_test_split(df, test_size=0.1, random_state=42)

In [21]:
all_training_words = [word for tokens in data_train["Tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["Tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

233627 words total, with a vocabulary size of 27185
Max sentence length is 858


In [22]:
all_test_words = [word for tokens in data_test["Tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test["Tokens"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

29346 words total, with a vocabulary size of 8175
Max sentence length is 346


In [23]:
print("Total Data Training : ", len(data_train))
print("Total Data Testing  : ", len(data_test))

Total Data Training :  3929
Total Data Testing  :  437


In [0]:
from gensim.models import Word2Vec

In [0]:
model = Word2Vec(df['Tokens'], size=100, min_count=1)

In [26]:
model.wv.save_word2vec_format('modelakhir.txt', binary=False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [27]:
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(data_train["Text_Final"].tolist())
training_sequences = tokenizer.texts_to_sequences(data_train["Text_Final"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

Found 27049 unique tokens.


In [0]:
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [29]:
print(train_cnn_data)

[[   0    0    0 ... 3211 1610  576]
 [   0    0    0 ...    0    0  291]
 [   0    0    0 ...  953 2670 4771]
 ...
 [   0    0    0 ...  343  178  177]
 [   0    0    0 ...   85   35  100]
 [   0    0    0 ...   20  695 2669]]


In [0]:
test_sequences = tokenizer.texts_to_sequences(data_test["Text_Final"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [31]:
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(data_train['Kelas']))
print('Shape of Data Tensor:', train_cnn_data.shape)
print('Shape of Label Tensor:', labels.shape)

indices = np.arange(train_cnn_data.shape[0])
np.random.shuffle(indices)
train_cnn_data = train_cnn_data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * train_cnn_data.shape[0])

x_train = train_cnn_data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = train_cnn_data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('Total Data Training   : ', len(x_train))
print('Total Data Validation : ', len(x_val))


Shape of Data Tensor: (3929, 1000)
Shape of Label Tensor: (3929, 2)
Total Data Training   :  3144
Total Data Validation :  785


In [0]:
EMBEDDING_DIM = 100

In [0]:
from gensim import models

In [34]:
word2vec_path = 'modelakhir.txt'
word2vec = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=100):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = clean_comments['Tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

In [0]:
training_embeddings = get_word2vec_embeddings(word2vec, data_train, generate_missing=True)

In [37]:
train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

embedding_layer = Embedding(len(train_word_index) + 1,
                            EMBEDDING_DIM,weights=[train_embedding_weights],
                            input_length=MAX_SEQUENCE_LENGTH,trainable=True)

(27050, 100)


In [38]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_cov1= Conv1D(128, 5, activation='relu')(embedded_sequences)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(5)(l_cov2)
l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
l_pool3 = MaxPooling1D(35)(l_cov3)  # global max pooling
l_flat = Flatten()(l_pool3)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(len(macronum), activation='softmax')(l_dense)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("Simplified convolutional neural network")
model.summary()
cp=ModelCheckpoint('model_cnn.hdf5',monitor='val_acc',verbose=1,save_best_only=True)

Simplified convolutional neural network
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         2705000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 996, 128)          64128     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 199, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 195, 128)          82048     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 39, 128)           0         
_________________________________________________________________
conv1d_3 (Conv1D)  

In [0]:
import time

In [41]:
awal = time.time()

history=model.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=10, batch_size=32,callbacks=[cp])

akhir = time.time()

durasi = akhir-awal
print(durasi)

Train on 3144 samples, validate on 785 samples
Epoch 1/10
3144/3144 [==============================] - 100s 32ms/step - loss: 0.6439 - acc: 0.7226 - val_loss: 0.5895 - val_acc: 0.7631

Epoch 00001: val_acc improved from -inf to 0.76306, saving model to model_cnn.hdf5
Epoch 2/10
3144/3144 [==============================] - 100s 32ms/step - loss: 0.4730 - acc: 0.8028 - val_loss: 0.5964 - val_acc: 0.7987

Epoch 00002: val_acc improved from 0.76306 to 0.79873, saving model to model_cnn.hdf5
Epoch 3/10
3144/3144 [==============================] - 100s 32ms/step - loss: 0.4168 - acc: 0.8435 - val_loss: 0.4889 - val_acc: 0.8025

Epoch 00003: val_acc improved from 0.79873 to 0.80255, saving model to model_cnn.hdf5
Epoch 4/10
3144/3144 [==============================] - 100s 32ms/step - loss: 0.3582 - acc: 0.8610 - val_loss: 0.4444 - val_acc: 0.8280

Epoch 00004: val_acc improved from 0.80255 to 0.82803, saving model to model_cnn.hdf5
Epoch 5/10
3144/3144 [==============================] - 100s

In [0]:
fig1 = plt.figure()
plt.plot(history.history['loss'],'r',linewidth=3.0)
plt.plot(history.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves :CNN',fontsize=16)
fig1.savefig('loss_cnn.png')
plt.show()

In [0]:
fig2=plt.figure()
plt.plot(history.history['acc'],'r',linewidth=3.0)
plt.plot(history.history['val_acc'],'b',linewidth=3.0)
plt.axis([0,10,0,1])
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves : CNN',fontsize=16)
fig2.savefig('accuracy_cnn.png')
plt.show()

In [44]:
print(history.history['acc'])
print(history.history['val_acc'])
print(history.history['loss'])
print(history.history['val_loss'])

[0.72264631043257, 0.8027989821882952, 0.8435114503816794, 0.8610050890585241, 0.8959923664122137, 0.9141221374045801, 0.936704834605598, 0.9510178117048346, 0.964058524173028, 0.9704198473282443]
[0.7630573252204117, 0.79872611487747, 0.8025477709284254, 0.8280254779347949, 0.8484076435398904, 0.8305732485595023, 0.752866242266005, 0.8343949046863872, 0.8127388537309732, 0.8535031848652348]
[0.6438942345347417, 0.47299525839378515, 0.4167902953448793, 0.3581835936954003, 0.32237020931171095, 0.27694958818095333, 0.23025992907981835, 0.189527678021417, 0.16808307806679235, 0.15884609249643816]
[0.5895011157746528, 0.596441863524686, 0.48887815437499127, 0.444351521874689, 0.40778635535270547, 0.5206103844247806, 0.7093578379624969, 0.4655270049526433, 0.9457286701840201, 0.5523817735113156]


In [45]:
scores = model.evaluate(x_train, y_train, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 99.14%


In [0]:
model_json = model.to_json()
with open("model_cnn.json", "w") as json_file:
    json_file.write(model_json)

In [47]:
model.save_weights("model_cnn.h5")
print("Saved model to disk")

Saved model to disk


In [48]:
hasil_cnn = {
    'acc' : history.history['acc'],
    'val_acc' : history.history['val_acc'],
    'loss' : history.history['loss'],
    'val_loss' : history.history['val_loss'],
    'durasi' : durasi
}
print(hasil_cnn)

{'acc': [0.72264631043257, 0.8027989821882952, 0.8435114503816794, 0.8610050890585241, 0.8959923664122137, 0.9141221374045801, 0.936704834605598, 0.9510178117048346, 0.964058524173028, 0.9704198473282443], 'val_acc': [0.7630573252204117, 0.79872611487747, 0.8025477709284254, 0.8280254779347949, 0.8484076435398904, 0.8305732485595023, 0.752866242266005, 0.8343949046863872, 0.8127388537309732, 0.8535031848652348], 'loss': [0.6438942345347417, 0.47299525839378515, 0.4167902953448793, 0.3581835936954003, 0.32237020931171095, 0.27694958818095333, 0.23025992907981835, 0.189527678021417, 0.16808307806679235, 0.15884609249643816], 'val_loss': [0.5895011157746528, 0.596441863524686, 0.48887815437499127, 0.444351521874689, 0.40778635535270547, 0.5206103844247806, 0.7093578379624969, 0.4655270049526433, 0.9457286701840201, 0.5523817735113156], 'durasi': 1040.584235906601}


In [0]:
import json

In [0]:
with open('hasil_cnn.json', 'w') as json_file:
  json.dump(hasil_cnn, json_file)

In [51]:
predictions = model.predict(test_cnn_data, batch_size=1, verbose=1)

437/437 [==============================] - 5s 10ms/step


In [52]:
print(predictions)

[[1.00000000e+00 4.34687923e-14]
 [5.34993261e-02 9.46500659e-01]
 [8.32363546e-01 1.67636469e-01]
 [3.61617431e-02 9.63838279e-01]
 [3.45512351e-04 9.99654472e-01]
 [8.90357405e-05 9.99910951e-01]
 [1.99464979e-04 9.99800563e-01]
 [8.60679328e-01 1.39320701e-01]
 [1.00000000e+00 1.78830197e-11]
 [9.99974668e-01 2.53339531e-05]
 [9.97736394e-01 2.26360792e-03]
 [9.99208510e-01 7.91478727e-04]
 [9.98862028e-01 1.13795733e-03]
 [9.99390900e-01 6.09125535e-04]
 [1.57414307e-03 9.98425841e-01]
 [1.00000000e+00 1.40400913e-09]
 [9.99579370e-01 4.20637138e-04]
 [9.99967694e-01 3.23088971e-05]
 [1.96771184e-03 9.98032272e-01]
 [9.99865413e-01 1.34568647e-04]
 [6.97194846e-05 9.99930263e-01]
 [5.13755390e-03 9.94862437e-01]
 [9.99727190e-01 2.72832694e-04]
 [1.00000000e+00 1.39848958e-21]
 [1.00000000e+00 1.25856645e-14]
 [9.98926103e-01 1.07388769e-03]
 [9.98643517e-01 1.35646376e-03]
 [7.23302305e-01 2.76697695e-01]
 [9.96168911e-01 3.83108738e-03]
 [9.99468982e-01 5.30997175e-04]
 [9.998233

In [53]:
print(data_test[:2])

                                                 Konten  ... Kelas
1551   (1) http://bit.ly/2rhtadc / http://bit.ly/2mx...  ...     0
561    “lolongan histeris siksa kubur alexander mend...  ...     1

[2 rows x 4 columns]


In [0]:
class_category = ['HOAX','FAKTA']

In [76]:
for i in range(predictions.shape[0]):
    print(str(i)+" "+class_category[predictions[i].argmax()])

0 HOAX
1 FAKTA
2 HOAX
3 FAKTA
4 FAKTA
5 FAKTA
6 FAKTA
7 HOAX
8 HOAX
9 HOAX
10 HOAX
11 HOAX
12 HOAX
13 HOAX
14 FAKTA
15 HOAX
16 HOAX
17 HOAX
18 FAKTA
19 HOAX
20 FAKTA
21 FAKTA
22 HOAX
23 HOAX
24 HOAX
25 HOAX
26 HOAX
27 HOAX
28 HOAX
29 HOAX
30 HOAX
31 HOAX
32 HOAX
33 FAKTA
34 HOAX
35 HOAX
36 FAKTA
37 FAKTA
38 FAKTA
39 FAKTA
40 HOAX
41 HOAX
42 FAKTA
43 HOAX
44 HOAX
45 FAKTA
46 HOAX
47 HOAX
48 HOAX
49 FAKTA
50 HOAX
51 FAKTA
52 FAKTA
53 FAKTA
54 FAKTA
55 HOAX
56 FAKTA
57 FAKTA
58 HOAX
59 HOAX
60 HOAX
61 HOAX
62 FAKTA
63 FAKTA
64 HOAX
65 FAKTA
66 HOAX
67 FAKTA
68 FAKTA
69 HOAX
70 HOAX
71 HOAX
72 HOAX
73 HOAX
74 HOAX
75 FAKTA
76 HOAX
77 HOAX
78 HOAX
79 FAKTA
80 FAKTA
81 FAKTA
82 FAKTA
83 FAKTA
84 HOAX
85 HOAX
86 FAKTA
87 FAKTA
88 HOAX
89 FAKTA
90 FAKTA
91 FAKTA
92 FAKTA
93 HOAX
94 HOAX
95 FAKTA
96 FAKTA
97 FAKTA
98 HOAX
99 HOAX
100 FAKTA
101 FAKTA
102 HOAX
103 FAKTA
104 HOAX
105 HOAX
106 HOAX
107 FAKTA
108 HOAX
109 FAKTA
110 HOAX
111 HOAX
112 FAKTA
113 FAKTA
114 HOAX
115 FAKTA
116 HOAX
117 HO

In [0]:
labels = [0,1]

In [0]:
prediction_labels=[]
for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])

In [0]:
#prediction_labels

In [80]:
sum(data_test.Kelas==prediction_labels)/len(prediction_labels)

0.8924485125858124

In [81]:
print(sum(data_test.Kelas==prediction_labels))

390


In [82]:
print(sum(data_test.Kelas!=prediction_labels))

47


In [83]:
print(len(prediction_labels))

437
